In [ ]:
import numpy as np
import pathlib 

import matplotlib.pyplot as plt

import torch

from torch.utils.data import Dataset

import sys
sys.path.insert(0, '../')
import sdss_dataset_lib
import sdss_psf

from astropy.io import fits
from astropy.wcs import WCS

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import os

In [ ]:
# load data
bands = [2]
x0 = 630
x1 = 310
sdss_hubble_data = sdss_dataset_lib.SDSSHubbleData(x0 = x0,
                                                    x1 = x1, 
                                                    bands = bands, 
                                                  fudge_conversion = 1.0)

In [ ]:
plt.hist(np.log10(sdss_hubble_data.fluxes.squeeze()), bins = 50);

In [ ]:
# the full image
plt.matshow(sdss_hubble_data.sdss_image_full[0])

In [ ]:
# check the hubble coordinates overlap with the globular cluster
plt.matshow(sdss_hubble_data.sdss_image_full[0])
plt.plot(sdss_hubble_data.locs_full_x1, 
         sdss_hubble_data.locs_full_x0, alpha = 0.2)

In [ ]:
# check patch 

for i in range(len(bands)):
    plt.matshow(sdss_hubble_data.sdss_image[i])
    plt.colorbar()


In [ ]:
sky_intensity = sdss_hubble_data.sdss_background.reshape(len(bands), -1).mean(1)

In [ ]:
sky_intensity

In [ ]:
# check alignment between bands 

if len(bands) > 1: 
    band_diff = (sdss_hubble_data.sdss_image[1] - sky_intensity[1]) - \
                (sdss_hubble_data.sdss_image[0] - sky_intensity[0])

    plt.matshow(band_diff, vmax = band_diff.abs().max(), vmin = -band_diff.abs().max(), 
                cmap = plt.get_cmap('bwr'))

    plt.colorbar()

# Distribution of colors

In [ ]:
if len(bands) > 1: 
    foo = (sdss_hubble_data.sdss_image[1] - sky_intensity[1]) / \
                (sdss_hubble_data.sdss_image[0] - sky_intensity[0])

    foo = torch.log10(foo).flatten() * (-2.5)
    plt.hist(foo, bins = 100);

    print(foo.mean())
    print(foo.var().sqrt())

In [ ]:
if len(bands) > 1: 

    foo = (sdss_hubble_data.fluxes[:, 1]) / \
                (sdss_hubble_data.fluxes[:, 0])

    foo = torch.log10(foo).flatten() * (-2.5)
    plt.hist(foo, bins = 100);

    print(foo.mean())
    print(foo.var().sqrt())

# plot a few subimages

In [ ]:
fmin = 1000.

In [ ]:
import plotting_utils

In [ ]:
x0_vec = np.arange(0, 100, 10)
x1_vec = x0_vec

In [ ]:
for i in range(6): 
    x0 = int(np.random.choice(x0_vec, 1))
    x1 = int(np.random.choice(x1_vec, 1))
    
    which_bright = (sdss_hubble_data.fluxes > fmin)[:, 0]
    
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))
    plotting_utils.plot_subimage(axarr[0], 
                                sdss_hubble_data.sdss_image[0], 
                                None, 
                                sdss_hubble_data.locs[which_bright], 
                                x0, x1, 
                                subimage_slen = 10, 
                                add_colorbar = True, 
                                global_fig = f)
    axarr[0].set_title('band ' + str(bands[0]))
    
    if len(bands) > 1: 
        plotting_utils.plot_subimage(axarr[1], 
                                    sdss_hubble_data.sdss_image[1], 
                                    None, 
                                    sdss_hubble_data.locs[which_bright], 
                                    x0, x1, 
                                    subimage_slen = 10, 
                                    add_colorbar = True, 
                                    global_fig = f)
        axarr[1].set_title('band ' + str(bands[1]))

        plotting_utils.plot_subimage(axarr[2], 
                                    (sdss_hubble_data.sdss_image[1] - sky_intensity[1]) - \
                                     (sdss_hubble_data.sdss_image[0] - sky_intensity[0]), 
                                    None, 
                                    sdss_hubble_data.locs[which_bright], 
                                    x0, x1, 
                                    subimage_slen = 10, 
                                    add_colorbar = True, 
                                    global_fig = f, 
                                    diverging_cmap = True)

# Test my simulator

In [ ]:
import simulated_datasets_lib
import fitsio

In [ ]:
# load psf 
psf_dir = '../../multiband_pcat/Data/idR-002583-2-0136/psfs/'
psf_r = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-r.fits')[0].read()
psf_i = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-i.fits')[0].read()

psf_og = np.array([psf_r])

In [ ]:
simulator = simulated_datasets_lib.StarSimulator(psf = psf_og,
                                    slen = sdss_hubble_data.slen, 
                                    transpose_psf = False, 
                                    sky_intensity = sky_intensity * 0.0 + 823.)

In [ ]:
_fluxes = sdss_hubble_data.fluxes.unsqueeze(0)
_locs = sdss_hubble_data.locs.unsqueeze(0)
_n_stars = torch.Tensor([len(sdss_hubble_data.locs)]).type(torch.LongTensor)

In [ ]:
recon_mean = simulator.draw_image_from_params(locs = _locs, 
                                                fluxes = _fluxes, 
                                                n_stars = _n_stars, 
                                                add_noise = False) 

In [ ]:
for i in range(len(bands)): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))

    observed = sdss_hubble_data.sdss_image
    im0 = axarr[0].matshow(observed[i])
    f.colorbar(im0, ax=axarr[0])
    axarr[0].set_title('observed, band = ' + str(bands[i]))


    im1 = axarr[1].matshow(recon_mean[0, i])
    f.colorbar(im1, ax=axarr[1])
    axarr[1].set_title('recon, band = ' + str(bands[i]))

    residual = recon_mean[0,i] - observed[i]
    foo = (residual / observed[i])
    im2 = axarr[2].matshow(foo, vmax = foo.abs().max(), vmin = - foo.abs().max(), cmap = plt.get_cmap('bwr'))
    f.colorbar(im2, ax=axarr[2])
    axarr[2].set_title('recon - obse, band = ' + str(bands[i]))

In [ ]:
residual = (recon_mean[0,0] - observed[0])

In [ ]:
residual[5:95, 5:95].median()

In [ ]:
plt.hist(residual.flatten().clamp(min = -2000), bins = 100);

In [ ]:
1 / (1 - 0.12)

In [ ]:
### plot some subimages 
f, axarr = plt.subplots(1, 3, figsize=(16, 4))

x0_vec = np.arange(0, 100, 10)
x1_vec = x0_vec

x0 = int(np.random.choice(x0_vec, 1))
x1 = int(np.random.choice(x1_vec, 1))

print([x0, x1])

plotting_utils.plot_subimage(axarr[0], 
                            observed[0], 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f)


plotting_utils.plot_subimage(axarr[1], 
                            recon_mean[0, 0], 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f)

plotting_utils.plot_subimage(axarr[2], 
                            (recon_mean[0, 0] - observed[0]) / observed[0], 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f, 
                            diverging_cmap = True)

# Check distribution on image stamps

In [ ]:
import image_utils

In [ ]:
sdss_hubble_data.sdss_image.shape

In [ ]:
image_stamps = \
    image_utils.tile_images(sdss_hubble_data.sdss_image.unsqueeze(0),
                            subimage_slen = 9,
                            step = 2)

In [ ]:
image_stamps.shape

In [ ]:
tile_coords = image_utils.get_tile_coords(sdss_hubble_data.sdss_image.shape[-1], 
                                          sdss_hubble_data.sdss_image.shape[-1],
                                        subimage_slen = 9, 
                                          step = 2);

In [ ]:
sdss_hubble_data.fluxes.shape

In [ ]:
subimage_locs, subimage_fluxes, n_stars, is_on_array = \
    image_utils.get_params_in_patches(tile_coords,
                                      sdss_hubble_data.locs[sdss_hubble_data.fluxes[:, 0] > fmin].unsqueeze(0),
                                      sdss_hubble_data.fluxes[sdss_hubble_data.fluxes[:, 0] > fmin].unsqueeze(0),
                                      sdss_hubble_data.sdss_image.shape[-1],
                                      subimage_slen = 9,
                                      edge_padding = 3)

In [ ]:
from torch.distributions.poisson import Poisson

In [ ]:
poisson_distr = Poisson(rate = 0.5)

In [ ]:
x = np.arange(0, 7)
h = plt.hist(n_stars, x)

plt.plot(torch.Tensor(h[1]), 
            h[0].sum() * torch.exp(poisson_distr.log_prob(torch.Tensor(h[1]))), 
           marker = 'x', color = 'red')

In [ ]:
sdss_hubble_data.sdss_image_full[0]

In [ ]:
plt.matshow(sdss_hubble_data.sdss_image_full[0][1200:1300, 200:300])